# How to Create Wikipedia Content Classification web application in Python?

Hello Eveyone,

In this article, we are going to learn to create a Wikipedia Content Classification Application. Before we actually go into the theory and learn to create this application, let us quickly see how this application looks like and how it works. 

After, running the Final application, you will be greeted with the following web interface. 

![Wikipedia Content Classification Web Applicatioin](https://raw.githubusercontent.com/datageekrj/FiverrContentRepo/master/wikipedia%20Content%20Classification/intro.PNG?token=ANKECZZ4PT2CGSCGA3IKQUK6WUUN4)

Read each and everything that is getting displayed on the image above. 

You should notice that the `Enter URL` field in the image. This is a place where the actual URL of the wikipedia page needs to entered by the `USER` of the application (whoever is using the application). 

When, we submit some URL in the that field, the following process will happen in the backend of the script:
  * The `request` module will get the content of the page corresponding to the URL that the user has entered
  * The `BeautifulSoup Module` will make the soup object and will get all the text of that wikipedia page for that `URL`
  * The model that we have trained using `sklean` (we will learn how to train model in this tutorial) in the backend will be called on the scraped text and we will get the prediction.
  * Them, that prediction will get displayed in the application. 

Let us assume that we enter a URL of the Tom Cruise Wikipedia Page and Hit Submit, then the display will look something like:

![Wikipedia Content Classification Web Applicatioin - Prediction on Top Cruise Wikipedia Page](https://raw.githubusercontent.com/datageekrj/FiverrContentRepo/master/wikipedia%20Content%20Classification/tom.PNG?token=ANKECZ7XUV2BOVNE5R4LTQS6WUUQM)

Then, we can go back and check the prediction of another web pages. It is a fun application. Isn't it?

So, this is what we will be building in this tutorial. So, buckle up..

I hope this small discussion on how the wikipedia classification application works makes sense to you. Now, that we understand what this application is and what are we building, we can actually get down to the steps of building it actually. 



# TASK FOR YOU: 
## TASK NUMBER. 1 What do you think of a web application? Try to answer yourself it yourself before looking at the answer

### Answer

A web application is just a normal website with some special purpose. We can have a web application for booking flights, or to watch live Tv shows etcetra. 

# Steps to create this Application...

**Learning Objectives:**
  * Creating our own `data` using web scraping with *Beautiful Soup* 
  * Once data has been scraped and created, cleaning and labelling the data
  * Creating a Pipeline for the Model using Multinomial Naive Baye's Algorithm
  * Finally, using the pipeline model, create a flask application and predict the results

# STEP.1 GETTING THE DATA

## Importing the necessary Modules

Before we actually start the actual coding process, we need to first import the nessary modules that we will require in the building of the application.

**Before we import any modules, I have included one code that installs all the necessary modules for this tutorial.** 

If you are sure, you can skip running this line.

In [1]:
!pip install requests
!pip install pandas
!pip install numpy
!pip install bs4
!pip install sklearn
!pip install flask==0.12.2 
!pip install flask-ngrok

     |████████████████████████████████| 92kB 2.4MB/s 
  Found existing installation: Flask 1.1.2
    Uninstalling Flask-1.1.2:
      Successfully uninstalled Flask-1.1.2


Once, we have installed all the necessary modules, we can easily import them. 

Note, here that some modules that we have imported below are not installed above. Why is that?

# TASK FOR YOU:

## TASK NUMBER.2 Why do you think that some modules were not installed earlier but they have been imported below? Try to answer yourself it yourself before looking at the answer

### Answer

The answer to this question is that some modules comes pre built with python. Hence, they are not required to be installed

# Importing the modules

Now, we can import the necessary modules which we will be using to create the web application

In [0]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
import re
from flask import Flask,request
from flask_ngrok import run_with_ngrok

# Let us briefly explain the objective of each of the above module

**Module Explanation:**
  * `requests` is for getting the content of the *URL* 
  * `pandas` is for making a data frame of the text
  * `numpy` is just for doing some basic computation on the datasets
  * `bs4` for converting the content that we get from requests.get to soup object which makes it easy for querying the `HTML`
  * `pipeline` from `sklearn` is for building a pipeline of the model. 
  * `sklearn` for training the `MultinomialNB` and preprocessing of the data
  * `re` for some text parsing and regular expression function for data cleaning.
  * `flask` for creating the actual web application
  * `flast_ngrok` for running the web server using internet

Now, that we understand what each of the modules do that we just imported, we can start coding this application. 

First, let us create a genral function which will accept the url of any wikipedia page and is going to return the text content of that page. 

## Given any URL, the following function will return the text content of that wikipedia page. 

In [0]:
def get_wiki_data(url):
  page = requests.get(url)
  html = page.text
  soup = BeautifulSoup(html)
  content = soup.find_all("p")
  data = []
  for paragraph in content:
    data.append(paragraph.text)
  data = " ".join(data)
  return data

Before understanding what is actually going on here, let us first run this function to see if it works as expected. We can pass the URL of any wikipedia page and get the content (content means all of the text content) of that wikipedia page. Let us check this function for the Machine Learning Web Page URL [Machine Learning](https://en.wikipedia.org/wiki/Machine_learning)

In [4]:
get_wiki_data(r"https://en.wikipedia.org/wiki/Machine_learning")

'Machine learning (ML) is the study of computer algorithms that improve automatically through experience.[1] It is seen as a subset of artificial intelligence. Machine learning algorithms build a mathematical model based on sample data, known as "training data", in order to make predictions or decisions without being explicitly programmed to do so.[2][3]:2 Machine learning algorithms are used in a wide variety of applications, such as email filtering and computer vision, where it is difficult or infeasible to develop  conventional algorithms to perform the needed tasks.\n Machine learning is closely related to computational statistics, which focuses on making predictions using computers. The study of mathematical optimization delivers methods, theory and application domains to the field of machine learning. Data mining is a related field of study, focusing on exploratory data analysis through unsupervised learning.[4][5] In its application across business problems, machine learning is 

As expected, the above result shows that the text is from the wikipedia page of Machine Learning. 


# TASK FOR YOU

## TASK NUMBER.3 Run the above function `get_wiki_data()` with some of your favorite wikipedia pages below and see if you get the appropriate result. Try to answer yourself it yourself before looking at the answer

### Answer

In [5]:
get_wiki_data(r"https://en.wikipedia.org/wiki/supervised_learning")

'Supervised learning is the machine learning task of learning a function that maps an input to an output based on example input-output pairs.[1] It infers a function from labeled training data consisting of a set of training examples.[2]  In supervised learning, each example is a pair consisting of an input object (typically a vector) and a desired output value (also called the supervisory signal).  A supervised learning algorithm analyzes the training data and produces an inferred function, which can be used for mapping new examples. An optimal scenario will allow for the algorithm to correctly determine the class labels for unseen instances. This requires the learning algorithm to generalize from the training data to unseen situations in a "reasonable" way (see inductive bias).\n The parallel task in human and animal psychology is often referred to as concept learning.\n In order to solve a given problem of supervised learning, one has to perform the following steps:\n A wide range o

# Making sense of `get_wiki_data()` function


Now that we know the above function works, let us understand each component piece by piece. 

First line of the code states that `page = requests.get(url)`. This line gets the web page corresponding to the URL that we pass to the function `get_wiki_data(url)`. Then, by doing `page.text` we extract the text component of the response that we get. From that, `text` response, we make a soup object using the function `BeautifulSoup()`. Note here that the text that we extracted from the response is not the actual text content of the wikipedia page but rather it is the text version of the html extracted from the web page. 

The reason we pass the `HTML` to soup because this library makes it really easy to extract specific content using the `find_all` method. 

Let me show you how `find_all` works. 

Before that, let us learn to understand and read some basic `HTML` code. The `HTML` code of any wikipedia page can be `inspected` by hitting `CONTROL + SHIFT + I` on chrome and on safari `COMMAND + OPTION + I`. This inspect will open a new window on which you can view the `HTML` content of the corresponding web page that you are looking at. 

The following image shows the HTML Contents of Machine Learning Wikipedia Page. 

<p align = "center"><img src = "https://raw.githubusercontent.com/datageekrj/ForHostingFiles/master/wikipediaInspect.PNG" height = "500"></p>

One Important thing that I want all of you to notice and pay attention on is thatr selected or highligted Machine Learning text on the right. Notice closely that it is between this strange looking symbols `<p>` and `<\p>`. Now, these are called in `HTML` language paragraph tags. This is what we should be aware off. 

Now, next in the above `get_wiki_data` function, notice we do something like `soup.find_all("p")`. With this code, what we are essentially doing is finding all such paragrapg tags and storing the result (which is the list of all paragrapg tags) in content. 

Now, that we have a list of all the paragraph tags, then we can loop over each one of the paragraph tag and get the text inside the corresponding tag and append the result in `data` list.

Finally, we return the data after joining each text inside that list by seperating each text by space (`" "`)

I hope how this function works is understood by everyone. 

I encourage everyone to try out this function with many other wikipedia page URLs. I want you play with the above function before moving to other bit of code. 

## Pattern of URL in Wikipedia Pages.



Consider a situation where you had some doubt in any topic and you go to Wikipedia to read about the corresponding topic. In that case, what you will most probably do is search for the topic name with wikipedia. Consider, the topic is `Machine Learning`, then you will probably search on google `Machine Learning Wikipedia` and this would yield number of results, the first of which will most probably the wikipedia page corresponding to your query. Now, when you go to that page, what you will essentially be going to is the URL `https://en.wikipedia.org/wiki/Machine_learning`. 

What if the topic was `Supervised Learning`, then the corresponding URL will be `https://en.wikipedia.org/wiki/Supervised_Learning`. I want Eveyone of you to notice the pattern in the URL of the page you are trying to search. 

Also, I want you all to go right now to google and search some about some of your favorite topics on wikipedia and i want you to notice the following two things:

* If the topic is of only one word say x, then the corresponding URL will be `https://en.wikipedia.org/wiki/` and `x` concatenated or joined. Notice, that the first part which is `https://en.wikipedia.org/wiki/` will always be there. 

* If the topic is of more than one word, then each word will be joined by underscore (`_`) and a new word will be formed. For Example, in the `Machine Learning` example, the URL will be `https://en.wikipedia.org/wiki/Machine_Learning`
 
So, the key takeaway is that each Wikipedia URL, will have the initial URL as `https://en.wikipedia.org/wiki/` and the other part will depend on whether your search query is of one word or it is of more than two words.

Hence, we declare a variable called `wiki_initial` as follows:

# TASK FOR YOU

## TASK NUMBER.4 Declare a Variable called `wiki_initial` with the initial wikipedia URL that we expect? Try to answer yourself it yourself before looking at the answer

### Answer

In [0]:
wiki_intial = "https://en.wikipedia.org/wiki/"

# Getting the Data

Now, that we understand that to get the content of any wikipedia page we need to use the function `get_wiki_data(url)` which is going to take one argument as `url` which we just understood can be made easily by just appending the name of the topic to the `wiki_initial` variable name. Then, we will have the URL of the corresponding wikipedia page.

So, to Make a Wikipedia Content Classification Model, we need to first decide:
* What are the different topics we need?
* How many different examples (no of wikipedia pages) we need for each topic?

These two questions are very important as Machine Learning Model will be good only if we have sufficient number of examples (wikipedia articles) for each category. 

From the screenshot of the Final Application in the beginning, the following are the different topics that our Classification Model can classify a particular example to:
* Celebrity
* Statistics
* Chemistry
* Physics
* Economics
* Mathematics

Okay, so, we have the answer of the First question and that was `What are the different topics we need?`. So, we have total six number of topics. 

Now, we need tom decide  `How many different examples (no of wikipedia pages) we need for each topic?`. This question is not easy to answer. It depends on person to person. If you want your model to have really good accuracy (You don't want your model to get wrong in prediction of the category), then in that case you will probably need really good number of examples for each category. 

There can be another person who just want something working really quick. In that case, you can probably have few examples for each of the category above. 

In this tutorial, we are going to be building an application with 300-400 examples but the code is easily extendable and I encourage you to add more examples. Doing this will be a piece of cake. You just need to find more topics and add that to the topic list that we are going to see in a little moment later. 

Now, note here we are going to have 200-300 examples, that means we are going to be using that function `get_wiki_data(url)` 200-300 times and it will take your data usage and time. So, depending upon the speed of your Internet Connection, you will probably have to wait for some time to get your data ready.

Now, let us one by Collect data for each category. 

# Collecting data for the Celebrity Group

We have a function called `get_Wiki_data(url)` and with the help of this function, as discussed earlier, we can get the content of any wikipedia page with the corresponding URL. 

Also, we discussed for getting the URL of the corresponding web page, we just need to know about the name of the topic. 

So, if we have a list of names, we can loop through it and get the appropriate content. 

The first category is `Celebrity`. For getting some examples of celebrity category, we first need to have a list of names of celebrity.

For this list, what i have done is, I have created a function called `get_list_ofimdb(url)` and what this function will do is get the of names of top 50 most popular hollywoord actors. 

We call this function on `https://www.imdb.com/list/ls053501318/` which is the corresponding link for the web page. 

In [0]:
def get_list_of_imdb(url):
  html = requests.get(url).text
  content = BeautifulSoup(html)
  req_list = []
  for cont in content.find_all("div", {"class":"lister-item mode-detail"}):
    req_list.append(cont.find("h3").find("a").text)
  return req_list
actor_list_holly = get_list_of_imdb("https://www.imdb.com/list/ls053501318/")

To get the list, first we make a request using request module and save the text response to `html` and then pass this to `BeautifulSoup` to make the soup and once that is done, we make an empty list and loop through the content one by one to get the appropriate content. 

You might wonder, why did we use `{"class":"lister-item mode-detail"}`, to demonstrate you this, I will inspect the corresponding imdb page and attach the image corresponding to this field to show the reason we added this class. Notice the yellow highlighted field below:

<img src = "https://raw.githubusercontent.com/datageekrj/ForHostingFiles/master/imdbinspect.PNG">

The class of the highlighted div corresponds to the `lister-item-mode-detail`. This is why we get that corresponding class. 

Using the `find_all` method on the soup content, we get a list of all the divs, and then we loop through those divs and find the `h3` tag and inside the h3 tag, notice that the `a` tag inside this `h3` tag and that tag contains the actor name. You need to look at the right of the above image (the yellow highlighted part). 

Finally, once we get the required list, we return that list. 

Now, notice that we call the function on the `URL` `https://www.imdb.com/list/ls053501318/`. 



# TASK FOR YOU

## TASK NUMBER.5 Check the length of the list `actor_list_holly`

### Answer

In [8]:
len(actor_list_holly)

50

# TASK FOR YOU

## TASK NUMBER.6 Check the name of the first actor from the list actor_list_holly. Try to answer yourself it yourself before looking at the answer

Now, that we have the list of hollywoord actors, we can start to get the content corresponding to that. 

But first check the first actor name...

### Answer

In [9]:
actor_list_holly[0]

' Johnny Depp\n'

# Cleaning the hollywood's actors names
 As you might have seen the name of the actors contain some un-wanting characters like `'\n'`, `'\s'`, `'^_'`. 

 So, to get rid of that we loop through the name scraped from the imdb website and we replace that with an empty string `''`. 

Hence, we have a list of  names called `names_holly` which is a list of cleaned actors name. 

In [0]:
names_holly = []
for actor in actor_list_holly:
  name = actor
  name = re.sub("\n", "", name)
  name = re.sub("\s", "_", name)
  name = re.sub("^_", "", name)
  names_holly.append(name)

# Scraping the content of the hollwood actor's wikipedia page

Okay, now that we have the appropriate list of names, now we can use the `get_wiki_data(URL)` function and the `wiki_initial` variable that we declared earlier to get the text content of the wikipedia pages. 

# TASK FOR YOU

## TASK NUMBER. 7 Make a list `text_data_holly` which will have the scraped content from the wikipedia page using the function `get_wiki_data(url)` and `wiki_initial` variable. Try to answer yourself it yourself before looking at the answer

### Answer

In [11]:

text_data_holly = []
for name in names_holly:
  text_data_holly.append(get_wiki_data(wiki_intial+name))
len(text_data_holly)

50

# Making a Data Frame from the created `text_data_holly`

In [12]:
hollywood = pd.DataFrame()
hollywood["text"] = text_data_holly
hollywood["label"] = ["Celebrity" for i in range(len(names_holly))]
hollywood.head()

,text,label
0,"\n John Christopher Depp II (born June 9, 1963...",Celebrity
1,\n \n Alfredo James Pacino (/pəˈtʃiːnoʊ/; Ital...,Celebrity
2,\n \n Robert Anthony De Niro Jr. (/də ˈnɪəroʊ/...,Celebrity
3,"\n \n Kevin Spacey Fowler KBE (born July 26, 1...",Celebrity
4,\n \n Denzel Hayes Washington Jr. (born Decemb...,Celebrity


## For other Categories (other than celebrity)

Once we have the celebrity data from wikipedia, we now need to grab other categories of data and it turns out to be quite repetitive as we will have to first define a list of categories and then create data and further throw any bad data. So, there are three steps to it:

* Make a List of Topics for each of the Catgories
* Get wikipedia text data for each topic in that list
* Throw out any bad data (which means throw out any wikipedia page which might now exist).

These lists, I have prepared myself and the choice of topics are those that I could think of while creating them. Now, in case you want to have more examples (more than 200 - 300), you just have to come up with your own new topics and add those in their respective topic list. The remaining code will just work fine as the code has been generalised. Even if you want to grap 1000 wikipedia pages for each of the topics, then in that case also this code is going to work just perfectly fine considering you have the patience to sit while this scraps 1000s of wikipedia pages.

## Creating Two Help Functions

Now, since the way we approch the scraping part of remaining categories is quite repetitive for all the five categries (chemistry, physics, economice, mathematics, statistics), we will create two supporting functions `create_category_data(names, label)` and `throw_bad_data(names,label)`. These two functions will do the job for all the five remaining categories. 

Let us understand these two functions one by one. 

### `create_category_data(names, label)`

This function will accept the list of topics as `names` and a corresponding group name as `label`. For example, I have got a list of topics of chemistry as `chem`, then i will call this function as `create_category_data(chem, "Chemistry")`. Here, `"Chemistry"` is the name of the label for the corresponding list of topics. Similay, for physics, we might call it like `create_category_data(phys, "Physics")` and so on.

This function first loops through the list which is provided by `names` and gets the data for each wikipedia page by calling the function `get_wiki_data(url)` again and again. Here, URL is created by appending each topic name to the `wiki_initial`.

Once, we get that list of all text, we create a data frame using pandas function `pd.DataFrame()`. This data frame will have two columns `text` and `label`. The `text` column will hold the list of texts that we just scraped for each of the topic and the `label` column will have the corresponding label which is going to be same for each text.

Now, finally this function returns this data frame the we just created.

### `throw_bad_data(names, label)`

This function takes tha same arguments as the `create_category_data(names, label)`. 

First, it will call `create_category_data(names, label)` to get all the data (It will include both good and bad data). 

Now, you might be wondering that what do we mean by bad data? The list of topics that I createt, was essentially created by me without searching on internet. In this case, there might be a situation where a particular topic that I might have entered in this list does not have a corresponding wikipedia page. 

Now, what happens if we try to search for a word which does not have a corresponding wikipedia page. Then, in that case, this function `throw_bad_data(names, label)` will look for references of `may refer to:` text in all the collected text, and if it finds one, it will record its index and finally using pandas indexing, it removes those texts. 

The remaining data with all good wikipedia texts will be returned by this function.

In [0]:
def create_category_data(names,label):
  text_data = [get_wiki_data(wiki_intial+i) for i in names]
  data = pd.DataFrame()
  data["text"] = text_data
  data["label"] = [label for i in range(len(names))]
  return data

In [0]:
def throw_bad_data(names,label):
  data = create_category_data(names,label)
  ind = []
  for i,j in enumerate(data["text"]):
    if "may refer to:" in j:
      ind.append(i)
  irrelevant = data.index.isin(ind)
  data = data[~irrelevant]
  return(data)

## Now, let us call this `throw_bad_data(names, label)` for each of the five categories and list

In [15]:
chem = ["compounds", "atoms", "molecules", "ions", "reaction", 
			 "chemist", "substances", "laboratory", "particles",
			 "energy", "radioactive", "matter", "photon", "mass",
			 "atomic_nucleus", "ionization", "metallic", "covalent",
			 "periodic", "isotopes", "solids", "silica", "sulfur", 
			 "sodium", "carbon", "alloy", "avogadro", "liquid", "oxidation",
			 "chloride", "plasma", "hydronium", "quanta", "phonons", "hydrogen",
			 "sulfied", "kinetics", "hydroxide", "phosphate", "redox", "molar",
			 "molarity", "helium", "krypton"]
chemistry = throw_bad_data(chem,"Chemistry")
chemistry.head()

,text,label
1,\n An atom is the smallest constituent unit of...,Chemistry
2,\n A molecule is an electrically neutral group...,Chemistry
3,"\n An ion (/ˈaɪɒn, -ən/)[1] is an atom or mole...",Chemistry
4,"Reaction may refer to a response to an action,...",Chemistry
5,A chemist (from Greek chēm (ía) alchemy; repla...,Chemistry


Let us define similar lists of other topics such as Physics, Maths, Statistics and Economics.

In [0]:
phys = ["energy", "force", "astronomy", "sun", "moon", "stars",
		   "planets", "optic", "gravitation", "gravity", "thermodynamics", 
		   "electromagnetics", "theory_of_relativity", "photoelectric_effect",
		   "speed_of_light", "motion", "superconductivity", "higgs_boson",
		   "quantum", "quantum_mechanics", "supersymmetry", "neutrinos", "meissner_effect",
		   "magnetic_field", "fluxons", "electric_current", "geomagnetism", "inertia",
		   "newton's_laws_of_motion", "classical_mechanics", "kepler's_law","planet",
		   "thermonuclear", "fusion", "orbit", "ellipse"]

maths = ["arithmetic", "algebra", "geometry", "analysis", "set_theory",
		 "number_theory", "natural_numbers", "complex_numbers", "infinity",
		 "groups", "rings", "fields", "abstract_algebra", "linear_algenra",
		 "vector_spaces", "analytic_geometry", "optimization", "tensor_calculus",
		 "convex", "topology", "real_analysis", "differential_equations", "euclidean_geometry",
		 "hypotenuse", "theorem", "volumes", "calculus", "integral", "differentiation",
		 "limits", "riemann", "fourier_series", "prime_numbers", "measure",
		 "infinite_series", "metric_space", "sine", "taylor_series", "lebesgue",
		 "hilbert_spaces", "functional_analysis", "ordered_pair"]

stats = ["probability", "surveys_sampling", "experiments", "inference", "observational_study",
	     "data_analysis", "mean", "variance", "dispersion", "descriptive_statistics",
	     "null_hypothesis", "missing_data", "probability_theory", "sampling", "estimator",
	     "statistic", "covariance", "mean_squared_error", "consistency",
	     "maximum_likelihood", "least_squares", "power_test", "random", "bayesian",
	     "frequentist", "significance", "critical_region","correlation", "regression", 
	     "logistic_regression", "factor_analysis", "cojoint_analysis","parametric",
	     "time_series_analysis", "bootstrap", "markov_chain"]

eco = ["production", "distribution_(economics)", "consumption_(economics)", "microeconomics",
		"macroeconomics", "demand", "supply", "national_income", "deficit",
		"resources", "market_equilibrium", "oligopoly", "monopoly", "duopoly",
		"monopsony", "investment_goods", "public_goods", "opportunity_cost",
		"factors_of_production", "pareto_efficiency", "production_possiblity_frontier",
		"scarcity", "constraints", "income_effect", "substitution_effect",
		"marginal_utility", "marginal_cost", "elasticity", "marginal_revenue",
		"perfectly_competitive","game_theory","economic_growth", "money","balance_of_payments",
		"central_bank", "fiscal_policy", "monetary_policy", "monye_supply",
		"exchange_rates", "externalities"]

# TASK FOR YOU

## TASK NUMBER. 8 Do the same with lists phys,maths,stats and eco and name the resulting data as `physics`,`mathematics`,`statistics` and `economics`. Try to answer yourself it yourself before looking at the answer

### Answer

In [0]:
physics = throw_bad_data(phys,"Physics")
mathematics = create_category_data(maths,"Mathematics")
statistics = create_category_data(stats,"Statistics")
economics = create_category_data(eco,"Economics")

# Combining all of the above six data to one data frame

Now, that we have data for all of the six categories, we can now concatenate every data frame to one data and call it `main_data` and finally we look at that data by using the `.head()` method on the resulting data frame.  

For that we will call the `concat` method of the `pandas` and will pass all the data that we have created viz `hollywood`, `chemistry`, `physics`, `mathematics`, `statistics` and `economics`. 

In [18]:
main_data = pd.concat((hollywood,chemistry,physics,mathematics,statistics,economics),ignore_index=True)
main_data.head()

,text,label
0,"\n John Christopher Depp II (born June 9, 1963...",Celebrity
1,\n \n Alfredo James Pacino (/pəˈtʃiːnoʊ/; Ital...,Celebrity
2,\n \n Robert Anthony De Niro Jr. (/də ˈnɪəroʊ/...,Celebrity
3,"\n \n Kevin Spacey Fowler KBE (born July 26, 1...",Celebrity
4,\n \n Denzel Hayes Washington Jr. (born Decemb...,Celebrity


# STEP. 2 Preparing the data for Machine Learning Model

Now, that we have a data named `main_data` ready for building machine learning model, we can start preparing data for building the model. 

## Why do we need to prepare data?

The data that we have just scraped is in text and all the machine learning algorithms accept only numbers. So, we need a way to convert that text into some suitable numbers which makes sense. 

## Mapping the labels to numbers

First, we will map each of the categories to some suitable numbers. The best choice would be number from 0-5 for each of the respective labels. To do that, we create a dictionary called `label_mapper` and then use the map function of the data frame, map each of the label. Then, we print the `main_data` to kind of see what we have got. Now, we have three columns, one extra for `label_number`.  

In [19]:
label_mapper = {"Celebrity":0, "Chemistry":1, "Physics":2,
               "Mathematics":3, "Statistics":4, "Economics":5}
main_data["label_number"] = main_data["label"].map(label_mapper)
main_data.head()

,text,label,label_number
0,"\n John Christopher Depp II (born June 9, 1963...",Celebrity,0
1,\n \n Alfredo James Pacino (/pəˈtʃiːnoʊ/; Ital...,Celebrity,0
2,\n \n Robert Anthony De Niro Jr. (/də ˈnɪəroʊ/...,Celebrity,0
3,"\n \n Kevin Spacey Fowler KBE (born July 26, 1...",Celebrity,0
4,\n \n Denzel Hayes Washington Jr. (born Decemb...,Celebrity,0


## Text Cleaning and Stop Words Removal

The text that we scraped from the wikipedia has lot of things that we are not intrested in. We are only intrested in the words (plain text). Further, we don't want those words which do not have much significance in predicting the category. One example of these kind of words is `stopwords`. Words such as `the,in,that etc` are said to be stopwords as they are often in every piece of english text and do not contain much discriminative power. 

So, for this step we perform the following two steps:
* We remove the stopwords
* We extract only words from the text or in other words, we get rid of numbers, or any unnecessay text by using regular expression

Hence, we import a library called `nltk` which will give us access to most of the stopwords available out there. So, after importing `nltk`, we download the `stopwords` corpus and store the english stopwords to the `stop` variable. 

After storing these stopwords in a variable, we construct a function called `clean_text`, the objective of which is to get rid of unnecessary details. Only take words into consideration. 

### `clean_text(text)`

This function will take `text` as input and it will first split that text into words. Now, after splitting the text into words, we will look for those words only which are not stopwords using the `stop` list that we got from the `nltk` library. 

After removing the stopwords, we then join all the text again and then using regular expression pattern, we split that new text on `[^A-Za-z]` which essentially means non-word character. Then, we join the resulting list on space and thus the new text will be free from any non word text. Finally, the resultant text will be returned.

In [0]:
stop = ['i','me','my','myself','we','our','ours','ourselves','you',"you're","you've","you'll","you'd",'your','yours','yourself','yourselves',
 'he','him','his','himself','she',"she's",'her','hers','herself','it',"it's",'its','itself','they','them','their','theirs','themselves','what','which',
 'who','whom','this','that',"that'll",'these','those','am','is','are','was','were','be','been','being','have','has','had','having','do','does','did','doing','a',
 'an','the','and','but','if','or','because','as','until','while','of','at','by','for','with','about','against','between','into','through','during','before','after',
 'above','below','to','from','up','down','in','out','on','off','over','under','again','further','then','once','here','there','when','where','why','how','all','any','both',
 'each','few','more','most','other','some','such','no','nor','not','only','own','same','so','than','too','very','s','t','can','will','just','don',"don't",'should',
 "should've",'now','d','ll','m','o','re','ve','y','ain','aren',"aren't",'couldn',"couldn't",'didn',"didn't",'doesn',"doesn't",'hadn',"hadn't",'hasn',"hasn't",'haven',
 "haven't",'isn',"isn't",'ma','mightn',"mightn't",'mustn',"mustn't",'needn',"needn't",'shan',"shan't", 'shouldn',"shouldn't",'wasn',"wasn't",'weren',"weren't",'won',"won't",
 'wouldn',"wouldn't"]

In [0]:
def clean_text(text):
  text = text.lower()
  tokens = text.split()
  new = [word for word in tokens if word not in stop]
  text = " ".join(new)
  text = " ".join(re.split(r"[^A-Za-z]", text))
  text = re.sub(" +", " ", text)
  return(text)

Finally, we loop through the `text` column of the datasets and call the `clean_text(text)` function for each text and append the cleaned text in `cleaned` list. 

# TASK FOR YOU

## TASK NUMBER.9 MAKE a list `cleaned` using the `text` column of the main data by calling the `clean_text(text)` function in a loop. Then, assign this cleaned list to a new column named `clean_text` of the `main_data` data frame. Try to answer yourself it yourself before looking at the answer

### Answer

In [22]:
cleaned = []
for i in main_data.text:
  cleaned.append(clean_text(i))
main_data["clean_text"] = cleaned
main_data.head()

,text,label,label_number,clean_text
0,"\n John Christopher Depp II (born June 9, 1963...",Celebrity,0,john christopher depp ii born june american ac...
1,\n \n Alfredo James Pacino (/pəˈtʃiːnoʊ/; Ital...,Celebrity,0,alfredo james pacino p t i no italian pa t i n...
2,\n \n Robert Anthony De Niro Jr. (/də ˈnɪəroʊ/...,Celebrity,0,robert anthony de niro jr d n ro italian de ni...
3,"\n \n Kevin Spacey Fowler KBE (born July 26, 1...",Celebrity,0,kevin spacey fowler kbe born july american act...
4,\n \n Denzel Hayes Washington Jr. (born Decemb...,Celebrity,0,denzel hayes washington jr born december ameri...


# STEP.3 Building the Machine Learning Model

So far we have scraped the data from wikipedia, cleaned the data and now it is time to build the model. As we know that we cannot 

# TASK FOR YOU

## TASK NUMBER. 10 Extract the `clean_text` column to a new variable `X` and the `label_number` variable to new variable `y`. Try to answer yourself it yourself before looking at the answer

### Answer

In [0]:
X = main_data.clean_text
y = main_data.label_number

# Model Building


Now, that we have two variables `X` and `y` as `text` and `label_number`, we can get started with the model building process.  

There are three steps to the model building process:

* We cannot just feed the text to the model, there has be some way of transforming that text into numbers by some suitable transformations. So, we use `CountVectorizer()` of `sklearn` module to convert the text into counts of words or also known as `bag-of-word` representation.
* Once, we have `bag-of-word` representation, we can feed that to our algorithm which is `Multinomial Naive Bayes` algorithm but the count vectorizer is not free from faults. First, it gives unnecessary counts to some non-informative words. For example, if we have a certain word which appears in almost all of the text and that particular word will have most likely least predictive power and that word cannot differentiate between two sentences. Hence, we need something more powerful. Hence, in this step we use something which is known as tfidf transformer. Here, tfidf stands for term-frequency-inverse-document-frequency. Here, term-frequency is what its name represents. It is the number representing how many times a word appears. 
Next, we have inverse term frequency which gives us some idea about how important a word is with respect to the rest of the collection of documents. 

* Now, finally, we have Multinomial Naive Bayes algorithm which is implemented in the `MultinomialNB()` class of `sklean`.

One of the nice things about sklean is that it provides us with the `Pipeline()` class which accepts all the preprocessors and alorithms in a list and gives us a nice way of training model in a pipeline. 

So, we pass each of the class in the pipeline in a list. Here, each element in a list is a tuple taking two elements. The first the name of the argument of the pipeline and the second is the actual class.

In [0]:
pipeline = Pipeline([
    ("count_vect", CountVectorizer()),
    ("tfidf", TfidfTransformer()),
    ("clf", MultinomialNB())
])

# TASK FOR YOU

## TASK NUMBER. 11 Call the fit method of the pipeline on the two variables `X` and `y`. Try to answer yourself it yourself before looking at the answer

### Answer

In [25]:
pipeline.fit(X,y)

Pipeline(memory=None,
         steps=[('count_vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

# Creating Support Functions for Prediction From this model pipeline 


In particular, we are going to be creating two helping functions namely `from_url_to_predict_data(url)` and `get_prediction(url,model)`. Let us explain both of these functions one by one:

### `from_url_to_predict_data(url)`

This function is going to take a url and get the data from that url using the function that we created earlier called `get_wiki_data`. Once this data has been scraped, we clean that text using `clean_text(text)` function which you can find in the step 2 of preparing data. Then, finally we make a data using pandas will be created by passing that cleaned function to the `pd.Series` function.


### `get_prediction(url,model)`

This function will first get the data from the `from_url_to_predict_data(url)` function and then will predict using the `pipeline` model that we have trained on the main_data. Now, the prediction of the model will be a number and that needs to converted into a suitable label for the user to see. For this, we are going to loop through the `label_mapper` that we created earlier and the corresponding to the prediction result, the required label will be returned using this function. 

Both the function have been defined below:


In [0]:
def from_url_to_predict_data(url):
  text = get_wiki_data(url)
  cleaned_text = clean_text(text)
  data_for_prediction = pd.Series(cleaned_text)
  return data_for_prediction

def get_prediction(url,model):
  req_data = from_url_to_predict_data(url)
  pred = model.predict(req_data)
  return [name for name,code in label_mapper.items() if code == pred][0]

Now, that we have our model ready and also we have a way to get the prediction of the text, now it is the time to test our prediction on some of the wikipedia URLs. 

Remember, we didn't include `Machine Learning` wikipedia page in our training. Hence, let us first see what our model thinks about this wikipedia page.

In [27]:
get_prediction(r"https://en.wikipedia.org/wiki/Machine_learning", pipeline)

'Statistics'

It makes sense as out of the four topics, Machine Learning can either belong to statistics or mathematics. If we had also computer science as one of the categories , then we would have probably got the answer as Computer Science. 

Now, let us check for other wikipedia pages and see how our model that we trained performs.

# TASK FOR YOU

## TASK NUMBER.12 Check the function `get_prediction(url, pipeline)` with some 3-4 urls of your wikipedia pages of your favorite topics. Try to answer yourself it yourself before looking at the answer

Answers

In [28]:
[get_prediction(r"https://en.wikipedia.org/wiki/Pareto", pipeline),
 get_prediction(r"https://en.wikipedia.org/wiki/computer_science", pipeline),
 get_prediction(r"https://en.wikipedia.org/wiki/hardy", pipeline),
 get_prediction(r"https://en.wikipedia.org/wiki/ramanujan", pipeline),
 get_prediction(r"https://en.wikipedia.org/wiki/confidence_interval", pipeline),
 get_prediction(r"https://en.wikipedia.org/wiki/factor_analysis", pipeline),
 get_prediction(r"https://en.wikipedia.org/wiki/big_bang_theory", pipeline)]


['Economics',
 'Mathematics',
 'Mathematics',
 'Celebrity',
 'Statistics',
 'Statistics',
 'Physics']

# STEP.4 Building the web application

To build the web application, we are going to be using a framework in python called `flask`. It is one of the most lightweight frameworks out there. We have already imported the `Flask` class of the module, now let us initialize it and wrap it with `run_with_ngrok`. The reason that we are doing `run_with_ngrok` is because it will provide us with two links one for opening the web application locally and another for opening the web application on internet

Next step is to initialize our app. The `Flask` function accepts one argument called `__name__`. 

In [0]:
app = Flask(__name__)
run_with_ngrok(app)

# How does a any web application work?

To learn about web application development, we need to know about the `HTML` and `CSS` which are two of the most important languages of web. 

We are going to talk about `HTML` and `CSS` in the next cell but right now I want to discuss how the flask app works.

To make a flask app, there are following steps:

* First make an `app` variable which is done in the above cell. 
* Then define the routes for app. For each route, we need to declare a function. In the later part of this tutorial, you will notice that we have made a function called `index()` before which we do something like `app.route("/")`. This is a python function decorator. It means that it tells python that the function which proceeds after this decorator is a special function and in this case it is a route function which will be extecuted once user tries to access our application

* Then, we have to decide what we want to show a user. The look and feel of the application is decided by `HTML` and `CSS`. 

* Them,once route function is set, we have to run the application using `app.run()`


* A very simple example of a very simple flask app which will give you some idea.

`from flask import Flask`

`app = Flask(__name__)`

`@app.route('/')`

`def hello_world():`

    `return 'Hello, World!'`

`if __name__ == "__main__":`

    `app.run()`

In this veri simple and minimal application, once the server has started then the user will be greeted with a simple text "Hello, World". Now, we can put `HTML` code in the return statement but that would make the function very difficult to read, and this why we have clearly separated the string of the `HTML`. 

I hope this small discussion on how flask web app works makes sense.

Now, let us understand a little bit about `HTML`.


## Making sense of the following `html_style` variable. 

First of let us understand what the heck is this long string?

If you are familiar with `HTML`, then the following string is just a piece of cake for you. But even if you are not, then also after reading this small discussion on `HTML` everything will start to make sense. 

### HTML - The language of Web

HTML stands for `Hyper Text Markup Language` and it is the language of the web. With the help of HTML, we can define the structure of the web application that we want to build. 

`HTML` decides the structure of a web page. Whatever you see on the web page is structured because of `HTML`. Now, everything on our `wikipedia content classification application` is also structured using `HTML`.

But Note here, HTML only handles the structure of the application while the look and feel of your application is handled by another technique called `CSS`. More on that later. 

To understand the following string, first of all understand that everything in `HTML` is made up of tags. 

What are tags? You might wonder..

Tags in HTML are made up of opening and closing tags. Opening tags looks like `<tag_name>` and closing tags looks like `</tag_name>`. 


Let us tage the example of our application. It has a input box, a submit button, various texts. The first is the header of the application which is handled by header tags. There can be various different types of header tags. 

* `h1` header tag
* `h2` header tag
* `h3` header tag
* `h4` header tag
* `h4` header tag


These all are used to place header for the HTML content. Each of them differs from each other because of the the size. 

Taking `h1` as an example, the opening tag would look like `<h1>` and the closing tag would look like `</h1>`. The content will be placed in between. 

Now, there are some types of tags which are self closing which means for those tags, there are no closing tags. `input` tag is one of the example self closing tag. 

### One special tag `<style> </style>`

If you observe the following `html_style` text below, most of the content reside in between `<style>` and `</style>`. If you don't notice this immediately, I want all of you to stop right now and notice this right now. Then, you can move on.




# Understanding HTML and CSS with example

`HTML` and `CSS` are very simple to learn and master. I will give you a very simple example to explain a few concepts. 

`HTML` 

Html has two main tags `<head></head>` and `<body></body>`. Notice, there is no body tag in the following `html_style` variable. The reason is because the body is what gets displayed to users and that part is explained in later article called `Two States of Application`

Then, what is the use of this `<head></head>` tag. It turns our that this head tag holds those things that user does not see but they have a major impact on any web page. 


One of the important things that this head tag holds is `<style></style>` tag. 

This tag holds all the color, size, display, shadow, radius of HTML elements etc. 

This style tag holds something which is known as CSS, `casacading style sheets`, it enables the web application to have the look that we want. 

In CSS, we have a concept of selectors and those selectors enables us to style individual selections. 

For example, I want to style the `<h1></h1>` tag, then I can select that tag in CSS as 

h1 {
I can put all the different style property values to the h1 HTML tag.
}

In this similar manner, I can style all the HTML tags. 

If you want, you can also change some of the properties that are written below to see the effect of those on the final application. 


Now, you might be wondering do I have to memorise all these rules?

The answer is `No`. The reason is that they are huge in numbers and all you need to do is type what style you are looking for in the google search bar and the corresponding style will come up as a first page on the internet. 

You need to learn these `HTML tags and CSS rules` only if you are really intrested about getting into web development. 

But on the other hand, if you are just looking for creating machine learning web applications, then you don't have to learn these rules of HTML and CSS but if you could, that would be really helpful and you will be able to create wonderful web applications. 

In [0]:
html_style = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>New Style</title>
    <style>
        body,html,*{
            margin:0px;
            padding: 0px;
        }
        .nesto-background{
            background: url(https://dtsvkkjw40x57.cloudfront.net/1350xnull/8160/uploads/2ab85f0f-bc62-417f-8a33-a064535a6ebd.jpg);
            opacity: 0.9;
            height: 300px;
            width: 100%;
            margin: 0px 0px 10px 0px;
            display: flex;
            flex-direction: column;
            justify-content: center;
            align-items: center;
        }
        .nesto-background div{
            align-self: flex-end;
        }

        h1 {
            border: 0px solid #F49200;
            background-color: #F49200;
            border-radius: 5px;
        }
        .remaining{
            display: flex;
            flex-direction: row;
        }
        .form{
            width: 60%;
            align-self: flex-start;
            margin-top: 100px;
        }
        .info{
            align-self: flex-end;
            display: flex;
            width:40%;
            flex-direction: column;
            justify-content: center;
            align-items: center;
            margin-top: 100px;
            margin-left:15px;
        }
        
        form{
            display: flex;
            flex-direction: column;
        }
        input[name=url]{
            width: 200px;
            height: 25px;
            margin-top: 20px;
            align-self: center;
        }

        input[name=submit]{
            width: 100px;
            height: 30px;
            outline: none;
            border: none;
            background-color: #F49200;
            color: white;
            margin-top: 10px;
            align-self: center;
        }

        h4{
            font-size: 20px;
            color: #F49200;
        }
        h3{
            color: red;
            margin-top: 10px;
        }
        ul{
            margin-top: 10px;
        }
        li{
            margin-left: 30px;
        }
        h2{
            color: #F49200;
        }
    </style>
</head>
'''

## Two states of our application - Posted or not Posted

The final application will be having two different states. One, where the user of the application will be shown the input box and some information about the application. The another state is where the user will submit the `URL` and tries to get the prediction that `URL`. 

So, for each of these states, I have different `HTML` string. One which is shown when the User visits the application `html_string_not_posted` and one where the user posts some URL and is expecting some return of prediction and for that notice we have created a function called `get_html_posted_string(data)` which accepts the data argument and that data is nothing but the prediction of the `URL` posted by user.

Notice one thing, we add html_style to the beginning in both the states (`html_string_not_posted` and in the function `get_html_posted_string(data)`). It is because the application style (which is color, font size, and all other styles are consistent for both the states of the application). 

Hence, although the user might be looking at the welcome screen of the application or he might be looking at the screen where the result is displayed after he submitted his `URL`, for both of these states, the style should be same and that style we declared earlier as `html_style`.

Now, remember the screen which is displayed once the user opens the app?

This is the state when the `input` is shown. In that page, there were lot of things. Like text `Welcome to the Wkipedia Content Classification Application` and other texts. Also, we had list of categories informing users about the six categroies. 




In [0]:
html_string_not_posted = html_style + '''
<body>
    <div class="nesto-background">
        <h1> Welcome to the Wikipedia Content Classification Application</h1>
        <div> <img src = "https://raw.githubusercontent.com/datageekrj/ForHostingFiles/master/NestoTV_1280x720%20(1).png" width = "200" height = "80" style = "align-self:center;"></div>
    </div>
    <div class="remaining">
        <div class="form">
            <h4>To use this application, Just Enter the URL of the wikipedia Page that you want to classify and hit submit..</h4>
            <form method="POST">
                <input type="text" name="url" class="url" placeholder="Enter URL">
                <input type="submit" name="submit" class="submit">
            </form>
        </div>
        <div class="info categories">
            <h2>Categories Included while Training the Naive Bayes Algorithm: </h2>
                <ul>
                    <li>Celebrity</li>
                    <li>Chemistry</li>
                    <li>Physics</li>
                    <li>Statistics</li>
                    <li>Mathematics</li>
                    <li>Economics</li>
                <h3> Data Source: Data Was Scraped Using Wikipedia by Python using BeautifulSoup</h3>
                </ul>
        </div>
    </div>
</body>
</html>
'''


Whereas on the other hand, once the user is displayed the prediction, there are only two things. 

* One is the `h1` tag which displays tha prediction and 
* another is a `a` tag for going back to the first state of the application. 

In [0]:
def get_html_posted_string(data):
  required_string = html_style + '''
<body>
    <div class="nesto-background">
        <h1> Welcome to the Wikipedia Content Classification Application</h1>
        <div> <img src = "https://raw.githubusercontent.com/datageekrj/ForHostingFiles/master/NestoTV_1280x720%20(1).png" width = "200" height = "80" style = "align-self:center;"></div>
    </div>
    <div class="remaining">
        <div style = "display:flex;flex-direction:column;align-items:center;justify-content:center;">
            <h1>Your URL that you have entered belongs to: <span style="color: #707070;">''' + data + '''
</span></h1><br>
        <a href="/">Go Back</a>
        </div>
        <div class="info categories">
            <h2>Categories Included while Training the Naive Bayes Algorithm: </h2>
                <ul>
                    <li>Celebrity</li>
                    <li>Chemistry</li>
                    <li>Physics</li>
                    <li>Statistics</li>
                    <li>Mathematics</li>
                    <li>Economics</li>
                <h3> Data Source: Data Was Scraped Using Wikipedia by Python using BeautifulSoup</h3>
                </ul>
        </div>
    </div>
</body>
</html>'''
  return required_string

## Creating the 'route' for our web application

Now, that we have defined the necessry html structure, we need to create the route of the web application. 

### What is route?

Route is just a fancy word for saying `URL`. Now, the `URL` which the user is going to enter in the input box is different from this URL. 

A web application is just like any other website you visit online. Now, when you visit any website you enter some `URL` and that URL is what we are talking about here.

So, since we want our web application to run on our own system. We don't want to share it with world. So, we have to create some way to open the web application on our local system. 

Hence, we create route for this purpose. 

So, before the function `index()`, we have a statement which starts with `@` which has a special name in python `decoraters`. 

Inside this route we say that the URL is `'/'` and we will accept two types of methods `GET` and `POST`. 

### Now, let us understand what do we mean by methods?

Remember, when we talked about the states of the application. There were two states:

* Where you are welcomed to enter the `URL` == `GET` request
* Once the user enters the `URL`, he will be displayed the prediction == `POST` request

Also, if you remember, we had two different types of string for both of the different types methods. For `GET` requests, we had `html_string_not_posted` while for the `POST` request we had a function which will take the `prediction` and give us the corresponding `HTML` representation. 

Now, inside the `index()` function, we then test whether the type is 'POST' or it is 'GET'. 


If the method is not `POST`, in the else block, we just return the `html_string_not_posted` while in the post, we first get the `url` that the user has entered using the `request.form.get` method. Once, the `URL` has been obtained, we call the `get_prediction()` function with the `url` and the `pipeline` that we trained using the scraped data from the wikipedia. 

Once, we get the prediction from the function, we store the result in `prediction`and then the function for second state of the application will be called and that is `get_html_posted_string` and the resulting `HTML` representation will be returned and consequently returned.  



In [0]:
@app.route('/', methods = ["GET", "POST"])
def index():
    if request.method == "POST":
        url = request.form.get("url")
        prediction = get_prediction(url, pipeline)
        required_posted_string = get_html_posted_string(prediction)
        return required_posted_string
    else:
        return html_string_not_posted

# Running the application

Finally, once we have set up the `HTML` string and the route, it is time to run the application by calling the method `run()` on the app variable that we declared. It is going to start the `Flask Server` and it is going to point you to a local url [http://127.0.0.1:5000/](http://127.0.0.1:5000/). 

In [0]:
if __name__ == "__main__":
  app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://68b45824.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [01/May/2020 09:48:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2020 09:48:20] "GET /favicon.ico HTTP/1.1" 404 -


After running the above piece of code, the server will be started and it will point you to two URLs. First, a http://127.0.0.1:5000/ which is a local host URL and it will only work if you are running the web application on your own machine . 

If you are running on the Cloud, then the that case use the URL with `ngrok` in it and it will take you to a new page where you can play with the web application

**Hence, this completes our discussion on creating a wikipedia classification web application using Flask and Wikipedia We Scraping Using Beautiful Soup and requests. This newly acquired knowledge, you can utilise in many areas such as building some other types of web application. You just have to follow the same thought process.**